In [1]:
import pandas as pd
import re

In [2]:
source = r'.\测试.xlsx'# 文件路径
out = r'.\结果.xlsx' # 输出结果路径

In [3]:
Original = pd.read_excel(source,sheet_name='Sheet1',dtype=str).fillna('')
Original

,No.,Vehicle W/ Engine
0,1,Dodge Challenger 2018-2023 6.2L V8 GAS OHV Sup...
1,2,Buick Regal 2011-2013 2.0L L4\nBuick Verano 20...
2,3,Lexus GS350 2007-2015 3.5L V6\nLexus GS450h 20...
3,4,Lexus GS350 2007-2015 3.5L V6\nLexus GS450h 20...
4,5,Buick Envision 2017-2020 2.5L L4\nBuick LaCros...
...,...,...
360,361,Porsche 911 2005-2008\nPorsche Boxster 2007-20...
361,362,Porsche 911 2012-2016\nPorsche Boxster 2013-20...
362,363,Porsche Boxster 2013-2016\nPorsche Cayman 2014...
363,364,Porsche 718 Boxster 2020-2024\nPorsche 718 Cay...


In [4]:
#将单条车型进行拆分
new_data = Original[['No.','Vehicle W/ Engine']].copy()
new_data['Vehicle W/ Engine'] = new_data['Vehicle W/ Engine'].str.split('\n')
new_data = new_data.explode('Vehicle W/ Engine')
clean_car=list(new_data['Vehicle W/ Engine'])
number=list(new_data['No.'])
new_data

,No.,Vehicle W/ Engine
0,1,Dodge Challenger 2018-2023 6.2L V8 GAS OHV Sup...
0,1,Dodge Charger 2021-2023 6.2L V8 GAS OHV Superc...
1,2,Buick Regal 2011-2013 2.0L L4
1,2,Buick Verano 2013-2016 2.0L L4
1,2,Chevrolet Cobalt 2010 2.0L L4
...,...,...
362,363,Porsche Cayman 2014-2016
363,364,Porsche 718 Boxster 2020-2024
363,364,Porsche 718 Cayman 2020-2024
364,365,Suzuki Forenza 2004-2005


In [5]:
#先输出单条车型进行拆分
import datetime
new_data.to_excel(out.replace('NOW()', datetime.datetime.now().strftime("%Y%m%d_%H%M%S")), index=False)
df0 = pd.read_excel(out,sheet_name='Sheet1',dtype=str).fillna('')
df0

,No.,Vehicle W/ Engine
0,1,Dodge Challenger 2018-2023 6.2L V8 GAS OHV Sup...
1,1,Dodge Charger 2021-2023 6.2L V8 GAS OHV Superc...
2,2,Buick Regal 2011-2013 2.0L L4
3,2,Buick Verano 2013-2016 2.0L L4
4,2,Chevrolet Cobalt 2010 2.0L L4
...,...,...
1917,363,Porsche Cayman 2014-2016
1918,364,Porsche 718 Boxster 2020-2024
1919,364,Porsche 718 Cayman 2020-2024
1920,365,Suzuki Forenza 2004-2005


In [6]:
# 创建DataFrame
df = pd.DataFrame(df0, columns=["No.", "Vehicle W/ Engine"])

In [7]:
# 通过正则表达式拆分数据
def split_vehicle_engine(text):
    # 提取车辆名称和发动机信息，限制年份在1900-2050，避免INTERNATIONAL 2554 1999 从2554开始切割
    #vehicle_match = re.findall(r'(.+?)(\d{4}-?\d{0,4})\s(.+)', text)
    pattern = r'(.+?)\s((?:19[0-9]{2}|20[0-4][0-9]|2050)(?:-(?:19[0-9]{2}|20[0-4][0-9]|2050))?)\s+(.+)'
    vehicle_match = re.findall(pattern, text)
    
    vehicles = []
    engines = []
    for match in vehicle_match:
        vehicles.append(f"{match[0]} {match[1]}".strip())  # 车辆名称和年份
        engines.append(match[2].strip())  # 发动机信息
    
    return "\n".join(vehicles), "\n".join(engines)

# 应用
df[['Vehicle W/0 Engine', 'Engine']] = df['Vehicle W/ Engine'].apply(lambda x: pd.Series(split_vehicle_engine(x)))

# 删除原始列
df.drop(columns=['Vehicle W/ Engine'], inplace=True)

# 按 No.进行分组并合并去重
df = df.groupby("No.", sort=True).agg({ 'Vehicle W/0 Engine': lambda x: "\n".join(sorted(set(x))), 'Engine': lambda x: "\n".join(sorted(set(x)))}).reset_index()

#No.按数字格式升序
df = df.sort_values(by="No.", key=pd.to_numeric, ascending=True)

df

,No.,Vehicle W/0 Engine,Engine
0,1,Dodge Challenger 2018-2023\nDodge Charger 2021...,6.2L V8 GAS OHV Supercharged
111,2,Buick Regal 2011-2013\nBuick Verano 2013-2016\...,2.0L L4
222,3,Lexus GS350 2007-2015\nLexus GS450h 2007-2011\...,3.5L V6\n4.6L V8\n5.0L V8
299,4,Lexus GS350 2007-2015\nLexus GS450h 2007-2011\...,3.5L V6\n4.6L V8\n5.0L V8
310,5,Buick Envision 2017-2020\nBuick LaCrosse 2018-...,2.5L L4
...,...,...,...
291,361,,
292,362,,
293,363,,
294,364,,


In [8]:
#处理双空格bug
df['Vehicle W/0 Engine'] = df['Vehicle W/0 Engine'].str.replace(r'\s{2,}', ' ', regex=True)
df['Engine'] = df['Engine'].str.replace(r'\s{2,}', ' ', regex=True)

In [9]:
#输出结果
df.to_excel(out.replace('NOW()', datetime.datetime.now().strftime("%Y%m%d_%H%M%S")), index=False)